In [2]:
import pandas as pd
from matplotlib import pyplot as plt
#!pip install pivottablejs
import seaborn as sns

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
latin = pd.read_stata('dataset/Latinobarometro_2020_Esp_Stata_v1_0.dta',convert_categoricals=False)
gpt = pd.read_csv('respuestas_gpt35_turbo.csv')
cohere = pd.read_csv('respuestas_cohere.csv')
bard = pd.read_csv('respuestas_bard.csv')

In [4]:
# Crea un StataReader para leer los metadatos
reader = pd.io.stata.StataReader('dataset/Latinobarometro_2020_Esp_Stata_v1_0.dta')

In [4]:
import pickle
with open(r"respuestas.pickle", "rb") as input_file:
    respuestas = pickle.load(input_file)
with open(r"preguntas_seleccionadas.pickle", "rb") as input_file:
    preguntas_seleccionadas = pickle.load(input_file)

In [5]:
preguntas_sin_orden = ['P33N','P34N']

In [6]:
import math
def calculate_distance(responde,llm):    
    result = 0
    num_pregs = 0
    for pregunta in list(preguntas_seleccionadas.keys()):
        if((pregunta.lower() in responde.keys()) and (not math.isnan(responde[pregunta.lower()])) and (pregunta in list(llm['Pregunta'])) 
           and(llm[llm['Pregunta']== pregunta]['Respuesta'].values[0] > 0)and (not pregunta in preguntas_sin_orden )):
            
            result = result + (abs(responde[pregunta.lower()]-llm[llm['Pregunta']== pregunta]['Respuesta'].values[0]))
            num_pregs = num_pregs + 1
    if(num_pregs == 0):
        result = -1
        norm_result = -1
    else:
        norm_result = result/num_pregs
    
    return(result, num_pregs, norm_result)
calculate_distance(latin.loc[0],bard)

(73.0, 58, 1.2586206896551724)

In [7]:
latin = latin[latin['idenpa']==32] #solo los argentinos

In [8]:
latin.shape

(1200, 408)

In [9]:
latin['bard_results'] = latin.apply(lambda x: calculate_distance(x, bard), axis=1)
latin[['bard_distance', 'bard_num_preg','bard_distance_norm']] = pd.DataFrame(latin['bard_results'].tolist(), index=latin.index)

In [10]:
latin['bard_num_preg'].value_counts()

bard_num_preg
58    1200
Name: count, dtype: int64

In [11]:
latin['gpt_results'] = latin.apply(lambda x: calculate_distance(x, gpt), axis=1)
latin[['gpt_distance', 'gpt_num_preg','gpt_distance_norm']] = pd.DataFrame(latin['gpt_results'].tolist(), index=latin.index)

In [12]:
latin['gpt_num_preg'].value_counts()

gpt_num_preg
24    1200
Name: count, dtype: int64

In [14]:
latin['cohere_results'] = latin.apply(lambda x: calculate_distance(x, cohere), axis=1)
latin[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(latin['cohere_results'].tolist(), index=latin.index)

C:\Users\PC-Usuario\AppData\Local\Temp\ipykernel_20376\3716045635.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latin['cohere_results'] = latin.apply(lambda x: calculate_distance(x, cohere), axis=1)
C:\Users\PC-Usuario\AppData\Local\Temp\ipykernel_20376\3716045635.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latin[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(latin['cohere_results'].tolist(), index=latin.index)
C:\Users\PC-Usuario\AppData\Local\Temp\ipykernel_203

In [15]:
latin.to_csv('latin_distances.csv')

In [16]:
# Crea un StataReader para leer los metadatos
reader = pd.io.stata.StataReader('dataset/Latinobarometro_2020_Esp_Stata_v1_0.dta')

In [25]:
latin.columns[0:40]

Index(['numinves', 'idenpa', 'numentre', 'reg', 'ciudad', 'tamciud', 'comdist',
       'edad', 'sexo', 'codigo', 'diareal', 'mesreal', 'ini', 'fin', 'dura',
       'totrevi', 'totcuot', 'totrech', 'totperd', 'numcasa', 'codsuper',
       'supervvi', 'superven', 'codif', 'digit', 'p1st', 'p2st', 'p3stgbs',
       'p4stgbs', 'p5stgbs', 'p6st', 'p7stgbs', 'p8st_a', 'p8st_b', 'p8st_c',
       'p9stgbs', 'p10stgbs', 'P11STGBS_A', 'P11STGBS_B', 'p12st'],
      dtype='object')

In [ ]:
['edad','sexo','P18ST','S19M','S9NPN','S24_A','S30','S16','S13','s17']

In [13]:
latin_ideo = latin[(latin['P18ST'.lower()]>=0) & (latin['P18ST'.lower()]<=10)]

In [14]:
latin_ideo.shape

(786, 416)

In [72]:
latin_ideo['s17'.lower()].value_counts()

s17
13    195
8     181
15    150
1      34
0      34
12     27
6      24
17     23
14     22
11     21
7      20
4      17
5      16
16      8
3       6
10      6
2       2
Name: count, dtype: int64

In [68]:
def convert(x):
    if(x == 14):
        return 16
    if(x== 15):
        return 17
    if(x == 16):
        return 14
    if(x==17):
        return 15
    return x
latin_ideo['s16_norm'] = latin_ideo['s16'].apply(lambda x: convert(x)) #terciario menos que universitario

C:\Users\PC-Usuario\AppData\Local\Temp\ipykernel_20376\2501243760.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latin_ideo['s16_norm'] = latin_ideo['s16'].apply(lambda x: convert(x)) #terciario menos que universitario


In [73]:
latin_ideo['s17_norm'] = latin_ideo['s17'].apply(lambda x: convert(x)) #terciario menos que universitario

C:\Users\PC-Usuario\AppData\Local\Temp\ipykernel_20376\187902359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latin_ideo['s17_norm'] = latin_ideo['s17'].apply(lambda x: convert(x)) #terciario menos que universitario


In [74]:
latin_ideo.to_csv('latin_ideo.csv')

In [7]:
reader.value_labels()['s17'.upper()]

{-5: 'No sabe / No contesta',
 -4: 'No preguntada',
 -3: 'No aplicable',
 -2: 'No contesta',
 -1: 'No sabe',
 0: 'Sin dato',
 1: 'Sin estudios',
 2: '1 aÃ±o',
 3: '2 aÃ±os',
 4: '3 aÃ±os',
 5: '4 aÃ±os',
 6: '5 aÃ±os',
 7: '6 aÃ±os',
 8: '7 aÃ±os',
 9: '8 aÃ±os',
 10: '9 aÃ±os',
 11: '10 aÃ±os',
 12: '11 aÃ±os',
 13: '12 aÃ±os',
 14: 'Universitario incompleto',
 15: 'Universitario completo',
 16: 'Instituto superior/academias/FormaciÃ³n tÃ©cnica incompleta',
 17: 'Instituto superior/academias/FormaciÃ³n tÃ©cnica completa'}

In [8]:
preguntas_seleccionadas['p18st'.upper()]

'En política se habla normalmente de "izquierda" y "derecha". En una escala dónde “00" es la “izquierda” y “10" la “derecha”, ¿Dónde se ubicaría Ud.?'

In [10]:
latin['sexo'].value_counts()

2    10537
1     9667
Name: sexo, dtype: int64